<h2> Christian Grier

In [91]:
%matplotlib inline
from pathlib import Path
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
import matplotlib.pylab as plt

In [92]:
DATA = Path("C:/Users/chris/Documents/DS 110-DM/data")

In [93]:
#loading dataframe
df = pd.read_csv(DATA / 'UniversalBank.csv')

<h3> Question 7.a

In [94]:
print(df.dtypes)

ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIP Code                int64
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal Loan           int64
Securities Account      int64
CD Account              int64
Online                  int64
CreditCard              int64
dtype: object


In [95]:
#new customer
newCustomer = pd.DataFrame([{'Age': 40, 'Experience': 10, 'Income': 84, 'Family': 2, 'CCAvg': 2, 'Mortgage': 0,
                             'Securities_Account': 0, 'CD_Account': 0, 'Online': 1, 'CreditCard': 1, 'Education_1': 0, 
                             'Education_2': 1, 'Education_3': 0}],
                            columns=['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account',
                                   'CD_Account', 'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3'])

In [96]:
#changing the data type of education to categorical
cols = ['Education']
df[cols] = df[cols].astype('category')

In [97]:
print(df.dtypes)

ID                       int64
Age                      int64
Experience               int64
Income                   int64
ZIP Code                 int64
Family                   int64
CCAvg                  float64
Education             category
Mortgage                 int64
Personal Loan            int64
Securities Account       int64
CD Account               int64
Online                   int64
CreditCard               int64
dtype: object


In [98]:
#correcting dataset and getting dummies
df = pd.get_dummies(df, prefix_sep='_', drop_first=False)
df.columns = [c.replace(' ', '_').replace('=', '_') for c in df.columns]

In [99]:
df.head()

,ID,Age,Experience,Income,ZIP_Code,Family,CCAvg,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,Education_1,Education_2,Education_3
0,1,25,1,49,91107,4,1.6,0,0,1,0,0,0,1,0,0
1,2,45,19,34,90089,3,1.5,0,0,1,0,0,0,1,0,0
2,3,39,15,11,94720,1,1.0,0,0,0,0,0,0,1,0,0
3,4,35,9,100,94112,1,2.7,0,0,0,0,0,0,0,1,0
4,5,35,8,45,91330,4,1.0,0,0,0,0,0,1,0,1,0


In [100]:
#creating the partitions
train_df, valid_df = train_test_split(df, test_size=0.4, random_state=1)
print('Training set:', train_df.shape, 'Validation set:', valid_df.shape)

Training set: (3000, 16) Validation set: (2000, 16)


In [101]:
#initializing the data set and transforming
#utilized the method in the book instead of the hint method
predictors = ['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 
                     'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3']
outcome = ['Personal_Loan']
scaler = preprocessing.StandardScaler()
scaler.fit(train_df[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 
                     'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3']])

# Transform the full dataset
bankNorm = pd.concat([pd.DataFrame(scaler.transform(df[['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 
                     'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3']]), 
                                    columns=['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities_Account', 'zCD_Account', 
                     'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3']),
                       df[['Personal_Loan']]], axis=1)
trainNorm = bankNorm.iloc[train_df.index]
validNorm = bankNorm.iloc[valid_df.index]
newCustomerNorm = pd.DataFrame(scaler.transform(newCustomer), columns=['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities_Account', 'zCD_Account', 
                     'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3'])
print(newCustomerNorm)

       zAge  zExperience   zIncome   zFamily    zCCAvg  zMortgage  \
0 -0.486446    -0.901063  0.220892 -0.352127  0.035689  -0.559242   

   zSecurities_Account  zCD_Account  zOnline  zCreditCard  zEducation_1  \
0            -0.337025    -0.252646  0.83419      1.53728     -0.838795   

   zEducation_2  zEducation_3  
0      1.591719     -0.660895  


In [102]:
#running the knn for new customer
knn = NearestNeighbors(n_neighbors=1)
# https://scikit-learn.org/stable/modules/neighbors.html#neighbors
knn.fit(trainNorm[['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities_Account', 'zCD_Account', 
                     'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3']])
distances, indices = knn.kneighbors(newCustomerNorm)
# kneighbors([X, n_neighbors, return_distance]) Finds the K-neighbors of a point.
# see https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
print(trainNorm.iloc[indices[0], :])  # indices is a list of lists, we are only interested in the first element

          zAge  zExperience   zIncome   zFamily    zCCAvg  zMortgage  \
4407 -0.747929    -0.639658 -0.059674 -0.352127 -0.136574  -0.559242   

      zSecurities_Account  zCD_Account  zOnline  zCreditCard  zEducation_1  \
4407            -0.337025    -0.252646  0.83419      1.53728     -0.838795   

      zEducation_2  zEducation_3  Personal_Loan  
4407      1.591719     -0.660895              0  


In [103]:
# Retrain with full dataset for 4 neighbors
bank_X = bankNorm[['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities_Account', 'zCD_Account', 
                     'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3']]
bank_y = bankNorm['Personal_Loan']
knn = KNeighborsClassifier(n_neighbors=1).fit(bank_X, bank_y)
distances, indices = knn.kneighbors(newCustomerNorm)
print(knn.predict(newCustomerNorm))
print('Distances',distances)
print('Indices', indices)
print(bankNorm.iloc[indices[0], :])

[0]
Distances [[0.47859833]]
Indices [[4034]]
          zAge  zExperience   zIncome   zFamily    zCCAvg  zMortgage  \
4034 -0.922251    -0.813928  0.177728 -0.352127 -0.136574  -0.559242   

      zSecurities_Account  zCD_Account  zOnline  zCreditCard  zEducation_1  \
4034            -0.337025    -0.252646  0.83419      1.53728     -0.838795   

      zEducation_2  zEducation_3  Personal_Loan  
4034      1.591719     -0.660895              0  


This customer would be classified as a non-successful customer. This can be seen by looking at the 0 prediction from the K-NN.

<h3> Question 7.b

In [104]:
#predicting the best knn for this dataset
train_X = trainNorm[['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities_Account', 'zCD_Account', 
                     'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3']]
train_y = trainNorm['Personal_Loan']
valid_X = validNorm[['zAge', 'zExperience', 'zIncome', 'zFamily', 'zCCAvg', 'zMortgage', 'zSecurities_Account', 'zCD_Account', 
                     'zOnline', 'zCreditCard', 'zEducation_1', 'zEducation_2', 'zEducation_3']]
valid_y = validNorm['Personal_Loan']

# Train a classifier for different values of k
results = []
for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results)
print(results)

     k  accuracy
0    1    0.9545
1    2    0.9500
2    3    0.9535
3    4    0.9495
4    5    0.9565
5    6    0.9495
6    7    0.9520
7    8    0.9460
8    9    0.9475
9   10    0.9435
10  11    0.9465
11  12    0.9435
12  13    0.9450
13  14    0.9435


The best K value that balances accuracy and over-fitting would be k = 5

<h3> Question 7.c

In [105]:
knn = KNeighborsClassifier(n_neighbors=5).fit(train_X, train_y)
print(knn.predict(newCustomerNorm))
knnPredict = knn.predict(valid_X)

[0]


In [106]:
knnPredict = knn.predict(valid_X)
confusion_matrix(valid_y, knnPredict)

array([[1803,    4],
       [  83,  110]], dtype=int64)

<h3> Question 7.d

I kind of already did this above but I will redo it here for easier grading.

In [107]:
knn.predict_proba(newCustomerNorm)

array([[1., 0.]])

This displays a 0 in predicted probability

<h3> Question 7.e

In [108]:
#creating the partitions
train_df, remaining_df = train_test_split(df, test_size=0.5, random_state=1)
valid_df, test_df = train_test_split(remaining_df, test_size=0.4, random_state=1)
print('Training set:', train_df.shape, 'Validation set:', valid_df.shape, 'Test:', test_df.shape)

Training set: (2500, 16) Validation set: (1500, 16) Test: (1000, 16)


In [109]:
#initializing dataset and transforming
#trying the method given in the question hints
predictors = ['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 
                     'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3']
outcome = ['Personal_Loan']
scaler = preprocessing.StandardScaler()
scaler.fit(train_df[predictors])

#Transform training and new customer
train_X = scaler.transform(train_df[predictors])
train_y = train_df[outcome]
valid_X = scaler.transform(valid_df[predictors])
valid_y = valid_df[outcome]
test_X = scaler.transform(test_df[predictors])
test_y = test_df[outcome]
test_X = pd.DataFrame(test_X, columns = ['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage', 'Securities_Account', 'CD_Account', 
                     'Online', 'CreditCard', 'Education_1', 'Education_2', 'Education_3'])
test_y = pd.DataFrame(test_y, columns = ['Personal_Loan'])
print(test_X)

          Age  Experience    Income    Family     CCAvg  Mortgage  \
0   -1.192185   -1.084066 -0.895312 -0.346955 -0.536477  0.466837   
1    1.513597    1.535886 -1.110618  0.522170 -1.053308  0.337320   
2   -1.017619   -0.909403 -0.701536 -1.216080 -0.346973 -0.559331   
3    0.117064    0.138578 -0.184801 -1.216080  0.267482 -0.559331   
4   -1.454035   -1.433393  1.300810 -0.346955  0.784312  1.473079   
..        ...         ...       ...       ...       ...       ...   
995 -1.104902   -1.171398  0.568770 -1.216080  0.439759  0.695981   
996 -0.581202   -0.647408  0.159688 -0.346955  0.497184 -0.559331   
997 -0.406635   -0.385412 -1.196740 -1.216080 -0.766180 -0.559331   
998 -0.319352   -0.298081 -0.313985  1.391296 -0.881031 -0.559331   
999 -0.668485   -0.734739 -0.744597 -0.346955 -0.938457  1.174195   

     Securities_Account  CD_Account    Online  CreditCard  Education_1  \
0             -0.352279    3.837091  0.833616    1.540720    -0.844691   
1             -0.352279

In [112]:
#finding results for best k
results = []
for k in range(1, 15):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results)
print(results)

<ipython-input-112-bf8055aa1239>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
<ipython-input-112-bf8055aa1239>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
<ipython-input-112-bf8055aa1239>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
<ipython-input-112-bf8055aa1239>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn = KNeighborsClassifier(n_nei

     k  accuracy
0    1  0.955333
1    2  0.951333
2    3  0.954667
3    4  0.948000
4    5  0.953333
5    6  0.942667
6    7  0.947333
7    8  0.943333
8    9  0.944000
9   10  0.938667
10  11  0.940667
11  12  0.940000
12  13  0.942000
13  14  0.938000


<ipython-input-112-bf8055aa1239>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)


In [114]:
knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(train_X, train_y)

<ipython-input-114-2c6544a9c9e0>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn.fit(train_X, train_y)


KNeighborsClassifier(n_neighbors=1)

In [122]:
knnPredTrain = knn.predict(train_X)
print(confusion_matrix(train_y, knnPredTrain))
print(accuracy_score(train_y, knnPredTrain))
knnPredValid = knn.predict(valid_X)
print(confusion_matrix(valid_y, knnPredValid))
print(accuracy_score(valid_y, knnPredValid))
knnPredTest = knn.predict(test_X)
print(confusion_matrix(test_y, knnPredTest))
print(accuracy_score(test_y, knnPredTest))

[[2259    0]
 [   0  241]]
1.0
[[1333   16]
 [  51  100]]
0.9553333333333334
[[897  15]
 [ 28  60]]
0.957


In this dataset we can see that the error rate decreases when it goes from the training to the validation set but it then increases again when it goes to the test set. This could be because of over-fitting but im not completely sure. 